In [10]:
%pip install opencv-python mediapipe

  Using cached mediapipe-0.10.32-py3-none-win_amd64.whl.metadata (9.8 kB)
Using cached mediapipe-0.10.32-py3-none-win_amd64.whl (10.2 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Face Dec

In [12]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

model_path = "blaze_face_short_range.tflite"

base_options = python.BaseOptions(model_asset_path=model_path)
options = vision.FaceDetectorOptions(
    base_options=base_options,
    running_mode=vision.RunningMode.VIDEO
)

detector = vision.FaceDetector.create_from_options(options)

cap = cv2.VideoCapture(0)
frame_count = 0

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)

    detection_result = detector.detect_for_video(mp_image, frame_count)
    frame_count += 1

    for detection in detection_result.detections:
        bbox = detection.bounding_box
        x, y, w, h = bbox.origin_x, bbox.origin_y, bbox.width, bbox.height

        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("Face Detection", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


# Face Landmark Dec

In [3]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

model_path = "face_landmarker.task"

base_options = python.BaseOptions(model_asset_path=model_path)

options = vision.FaceLandmarkerOptions(
    base_options=base_options,
    running_mode=vision.RunningMode.VIDEO,
    num_faces=1
)

landmarker = vision.FaceLandmarker.create_from_options(options)

cap = cv2.VideoCapture(0)
frame_count = 0

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)

    result = landmarker.detect_for_video(mp_image, frame_count)
    frame_count += 1

    if result.face_landmarks:
        for face_landmarks in result.face_landmarks:
            for landmark in face_landmarks:
                x = int(landmark.x * frame.shape[1])
                y = int(landmark.y * frame.shape[0])
                cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)

    cv2.imshow("Face Landmarks", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


# Drowsiness Detection

In [18]:
import cv2
import mediapipe as mp
import numpy as np
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

model_path = "face_landmarker.task"

base_options = python.BaseOptions(model_asset_path=model_path)

options = vision.FaceLandmarkerOptions(
    base_options=base_options,
    running_mode=vision.RunningMode.VIDEO,
    num_faces=1
)

landmarker = vision.FaceLandmarker.create_from_options(options)

cap = cv2.VideoCapture(0)
frame_count = 0

def euclidean(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

def get_eye_points(indices, landmarks, w, h):
    pts = []
    for i in indices:
        lm = landmarks[i]
        pts.append((int(lm.x * w), int(lm.y * h)))
    return pts

def compute_ear(points):
    vertical1 = euclidean(points[1], points[5])
    vertical2 = euclidean(points[2], points[4])
    horizontal = euclidean(points[0], points[3])
    return (vertical1 + vertical2) / (2.0 * horizontal)

# Left and Right eye indices
left_indices = [33, 160, 158, 133, 153, 144]
right_indices = [362, 385, 387, 263, 373, 380]

sleep_frame_counter = 0
SLEEP_THRESHOLD = 0.20
CONSEC_FRAMES = 15

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb)

    result = landmarker.detect_for_video(mp_image, frame_count)
    frame_count += 1

    if result.face_landmarks:
        face_landmarks = result.face_landmarks[0]
        h, w, _ = frame.shape

        left_eye = get_eye_points(left_indices, face_landmarks, w, h)
        right_eye = get_eye_points(right_indices, face_landmarks, w, h)

        left_ear = compute_ear(left_eye)
        right_ear = compute_ear(right_eye)

        ear = (left_ear + right_ear) / 2.0

        cv2.putText(frame, f"EAR: {ear:.2f}", (30, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Draw eye points
        for p in left_eye + right_eye:
            cv2.circle(frame, p, 2, (255, 0, 0), -1)

        # Sleep logic with consecutive frame check
        if ear < SLEEP_THRESHOLD:
            sleep_frame_counter += 1
        else:
            sleep_frame_counter = 0

        if sleep_frame_counter >= CONSEC_FRAMES:
            cv2.putText(frame, "SLEEPY", (30, 70),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Sleep Detection", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


# Also considering yawning

In [4]:
import cv2
import mediapipe as mp
import numpy as np
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

model_path = "face_landmarker.task"

base_options = python.BaseOptions(model_asset_path=model_path)

options = vision.FaceLandmarkerOptions(
    base_options=base_options,
    running_mode=vision.RunningMode.VIDEO,
    num_faces=1
)

landmarker = vision.FaceLandmarker.create_from_options(options)

cap = cv2.VideoCapture(0)
frame_count = 0

def euclidean(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

def get_eye_points(indices, landmarks, w, h):
    pts = []
    for i in indices:
        lm = landmarks[i]
        pts.append((int(lm.x * w), int(lm.y * h)))
    return pts

def compute_ear(points):
    vertical1 = euclidean(points[1], points[5])
    vertical2 = euclidean(points[2], points[4])
    horizontal = euclidean(points[0], points[3])
    return (vertical1 + vertical2) / (2.0 * horizontal)

def compute_mar(landmarks, w, h):
    top = landmarks[13]
    bottom = landmarks[14]
    left = landmarks[61]
    right = landmarks[291]

    top = (int(top.x * w), int(top.y * h))
    bottom = (int(bottom.x * w), int(bottom.y * h))
    left = (int(left.x * w), int(left.y * h))
    right = (int(right.x * w), int(right.y * h))

    vertical = euclidean(top, bottom)
    horizontal = euclidean(left, right)

    mar = vertical / horizontal

    return mar, [top, bottom, left, right]

# Eye indices
left_indices = [33, 160, 158, 133, 153, 144]
right_indices = [362, 385, 387, 263, 373, 380]

sleep_frame_counter = 0
CONSEC_FRAMES = 15

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb)

    result = landmarker.detect_for_video(mp_image, frame_count)
    frame_count += 1

    if result.face_landmarks:
        face_landmarks = result.face_landmarks[0]
        h, w, _ = frame.shape

        # EAR
        left_eye = get_eye_points(left_indices, face_landmarks, w, h)
        right_eye = get_eye_points(right_indices, face_landmarks, w, h)

        left_ear = compute_ear(left_eye)
        right_ear = compute_ear(right_eye)
        ear = (left_ear + right_ear) / 2.0

        # MAR
        mar, mouth_points = compute_mar(face_landmarks, w, h)

        # Display values
        cv2.putText(frame, f"EAR: {ear:.2f}", (30, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.putText(frame, f"MAR: {mar:.2f}", (30, 65),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

        # Draw eye points
        for p in left_eye + right_eye:
            cv2.circle(frame, p, 2, (255, 0, 0), -1)
        for p in mouth_points:
            cv2.circle(frame, p, 3, (0, 255, 255), -1)

        # ---- EXACT REQUESTED LOGIC ----
        if ear <= 0.15 or (ear <= 0.20 and mar >= 0.35):
            sleep_frame_counter += 1
        else:
            sleep_frame_counter = 0

        if sleep_frame_counter >= CONSEC_FRAMES:
            cv2.putText(frame, "SLEEPY", (30, 110),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

    cv2.imshow("Sleep Detection", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


# Testing Multiple windows at same time (to debug)

In [5]:
import cv2
import mediapipe as mp
import numpy as np
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# ------------------ LOAD MODEL ------------------
model_path = "face_landmarker.task"

base_options = python.BaseOptions(model_asset_path=model_path)
options = vision.FaceLandmarkerOptions(
    base_options=base_options,
    running_mode=vision.RunningMode.VIDEO,
    num_faces=1
)

landmarker = vision.FaceLandmarker.create_from_options(options)

# ------------------ CAMERA ------------------
cap = cv2.VideoCapture(0)
frame_count = 0

# ------------------ HELPERS ------------------
def euclidean(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

def get_eye_points(indices, landmarks, w, h):
    pts = []
    for i in indices:
        lm = landmarks[i]
        pts.append((int(lm.x * w), int(lm.y * h)))
    return pts

def compute_ear(points):
    vertical1 = euclidean(points[1], points[5])
    vertical2 = euclidean(points[2], points[4])
    horizontal = euclidean(points[0], points[3])
    return (vertical1 + vertical2) / (2.0 * horizontal)

def compute_mar(landmarks, w, h):
    top = landmarks[13]
    bottom = landmarks[14]
    left = landmarks[61]
    right = landmarks[291]

    top = (int(top.x * w), int(top.y * h))
    bottom = (int(bottom.x * w), int(bottom.y * h))
    left = (int(left.x * w), int(left.y * h))
    right = (int(right.x * w), int(right.y * h))

    vertical = euclidean(top, bottom)
    horizontal = euclidean(left, right)

    return vertical / horizontal

# ------------------ INDICES ------------------
left_indices = [33, 160, 158, 133, 153, 144]
right_indices = [362, 385, 387, 263, 373, 380]

sleep_counter = 0
CONSEC_FRAMES = 15

# ------------------ MAIN LOOP ------------------
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    raw_frame = frame.copy()
    landmarks_frame = frame.copy()
    sleep_frame = frame.copy()

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb)

    result = landmarker.detect_for_video(mp_image, frame_count)
    frame_count += 1

    if result.face_landmarks:
        face_landmarks = result.face_landmarks[0]
        h, w, _ = frame.shape

        # -------- LANDMARKS WINDOW --------
        for landmark in face_landmarks:
            x = int(landmark.x * w)
            y = int(landmark.y * h)
            cv2.circle(landmarks_frame, (x, y), 1, (0, 255, 0), -1)

        # -------- DROWSINESS WINDOW --------
        left_eye = get_eye_points(left_indices, face_landmarks, w, h)
        right_eye = get_eye_points(right_indices, face_landmarks, w, h)

        ear = (compute_ear(left_eye) + compute_ear(right_eye)) / 2.0
        mar = compute_mar(face_landmarks, w, h)

        cv2.putText(sleep_frame, f"EAR: {ear:.2f}", (30, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.putText(sleep_frame, f"MAR: {mar:.2f}", (30, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

        for p in left_eye + right_eye:
            cv2.circle(sleep_frame, p, 2, (255, 0, 0), -1)

        if ear <= 0.15 or (ear <= 0.20 and mar >= 0.35):
            sleep_counter += 1
        else:
            sleep_counter = 0

        if sleep_counter >= CONSEC_FRAMES:
            cv2.putText(sleep_frame, "SLEEPY", (30, 120),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

    # -------- SHOW ALL WINDOWS --------
    cv2.imshow("1 - Raw Camera", raw_frame)
    cv2.imshow("2 - All Landmarks", landmarks_frame)
    cv2.imshow("3 - Drowsiness Detection", sleep_frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
